In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

print(os.listdir('../input'))

In [ ]:
train_df =  pd.read_csv('../input/train.csv', nrows = 40_000_000)
test_df = pd.read_csv('../input/test.csv')
key = test_df['key'].values

In [ ]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()
    
def modify_date_time(string):
    return string.split(" ")[1].split(":")[0]

add_travel_vector_features(train_df)
train_df.pickup_datetime = train_df.pickup_datetime.apply(modify_date_time)
add_travel_vector_features(test_df)
test_df.pickup_datetime = test_df.pickup_datetime.apply(modify_date_time)

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))

In [ ]:
train_Y = train_df.fare_amount
train_X = train_df.drop(['key', 'fare_amount', 'dropoff_longitude', 'pickup_longitude', 'dropoff_latitude', 'pickup_latitude'], axis = 1)
test_X = test_df.drop(['key', 'dropoff_longitude', 'pickup_longitude', 'dropoff_latitude', 'pickup_latitude'], axis = 1)


In [ ]:
clf = Ridge(alpha=0.5)
clf.fit(train_X, train_Y)
predictions = clf.predict(test_X)

df = {'key' : key,'fare_amount' : predictions}
submission = pd.DataFrame(df)
submission.to_csv('taxi.csv', index = False)